#cupy

In [1]:
import cupy as cp
import numpy as np

ModuleNotFoundError: No module named 'cupy'

In [3]:
array_cpu = np.random.randint(0,255, size=(4000,4000)) # cpu array creation in ram

print("The array:", array_cpu)
print("\nMemery sizr:", array_cpu.nbytes/1e6, "MB")

The array: [[218  33 177 ... 167  53 239]
 [173  12 178 ...  36  84  98]
 [224  42 199 ... 157 215 120]
 ...
 [217  77 179 ... 183 251 171]
 [253 233 189 ... 207 110 135]
 [107  12  59 ... 128 144  22]]

Memery sizr: 64.0 MB


In [4]:
%%time
#sending the array to the gpu vram

array_gpu = cp.asarray(array_cpu)

print("The gpu array:",array_gpu)

#we have sent the array to the gpu

The gpu array: [[218  33 177 ... 167  53 239]
 [173  12 178 ...  36  84  98]
 [224  42 199 ... 157 215 120]
 ...
 [217  77 179 ... 183 251 171]
 [253 233 189 ... 207 110 135]
 [107  12  59 ... 128 144  22]]
CPU times: total: 62.5 ms
Wall time: 1.75 s


first fourier transform

In [5]:
%%time
from scipy import fft

fft.fftn(array_cpu) # cpu and ram involve

CPU times: total: 93.8 ms
Wall time: 344 ms


array([[ 2.03161333e+09     -0.j        ,
         3.26548938e+05-411655.66627035j,
        -1.58664517e+05+273442.56370014j, ...,
        -3.54724838e+05+424610.75696368j,
        -1.58664517e+05-273442.56370014j,
         3.26548938e+05+411655.66627035j],
       [-6.84232234e+03 +91117.92881636j,
         3.88697521e+05  -7645.73087923j,
         1.88184333e+05-170086.70233574j, ...,
        -3.77972949e+05+585784.96221418j,
        -9.41504408e+03 -17848.68004571j,
         3.05083456e+05 +23540.52055203j],
       [ 1.29028490e+05 +61145.18571746j,
        -1.74482497e+05 +19081.47777539j,
         3.90909867e+05+130302.86869298j, ...,
        -9.03565673e+04+255467.82583395j,
        -1.72915188e+05 +26230.85404958j,
        -6.28739748e+04 +23256.5041305j ],
       ...,
       [ 1.90172388e+05-118837.91090766j,
        -2.28429400e+05   -787.99787896j,
         3.68128488e+04 -76498.38303831j, ...,
        -3.03387712e+05 +93476.3828688j ,
         4.25460286e+04 -84663.51550928j,

In [6]:
%%time
from cupyx.scipy import fft as fft_gpu

fft_gpu.fftn(array_gpu) # cpu and

CPU times: total: 156 ms
Wall time: 468 ms


array([[ 2.03161333e+09     +0.j        ,
         3.26548938e+05-411655.66627035j,
        -1.58664517e+05+273442.56370014j, ...,
        -3.54724838e+05+424610.75696368j,
        -1.58664517e+05-273442.56370014j,
         3.26548938e+05+411655.66627035j],
       [-6.84232234e+03 +91117.92881636j,
         3.88697521e+05  -7645.73087923j,
         1.88184333e+05-170086.70233574j, ...,
        -3.77972949e+05+585784.96221418j,
        -9.41504408e+03 -17848.68004571j,
         3.05083456e+05 +23540.52055203j],
       [ 1.29028490e+05 +61145.18571746j,
        -1.74482497e+05 +19081.47777539j,
         3.90909867e+05+130302.86869298j, ...,
        -9.03565673e+04+255467.82583395j,
        -1.72915188e+05 +26230.85404958j,
        -6.28739748e+04 +23256.5041305j ],
       ...,
       [ 1.90172388e+05-118837.91090766j,
        -2.28429400e+05   -787.99787896j,
         3.68128488e+04 -76498.38303831j, ...,
        -3.03387712e+05 +93476.3828688j ,
         4.25460286e+04 -84663.51550928j,

In [7]:
#communicate host cpu and gpu


fft_cpu = fft.fftn(array_cpu)
fft_send_back = cp.asnumpy(fft_gpu.fftn(array_gpu))   #sending back to the cpu as numpy array

np.allclose(fft_cpu,fft_send_back)    #comparing the array of np and cp


True

In [8]:
import cupy as cp
import numpy as np
#at a glance communicating cpu and gpu

#step - 1: making a np array
array_cpu = np.random.randint(0,255, size=(4000,4000)) # cpu array creation in ram

#step - 2: sending the array to the gpu
array_gpu = cp.asarray(array_cpu)


#step - 3: makig fft with np array in gpu

from cupyx.scipy import fft as fft_gpu
fft_in_gpu = fft_gpu.fftn(array_gpu)


#step - : sending back to the cpu as numpy array
fft_send_back = cp.asnumpy(fft_in_gpu)


In [9]:
type(fft_send_back)

numpy.ndarray

In [10]:
#testing
fft_cpu = fft.fftn(array_cpu) #fft on cpu
np.allclose(fft_cpu,fft_send_back) 

True

In [11]:
#creating array dorect on gpu

array_gpu_direct = cp.random.randint(0,255, size=(4000,4000))
print(array_gpu_direct)

#convert it to numpy
array_numpy = cp.asnumpy(array_gpu_direct)

print(type(array_numpy))

[[128  94 194 ...  46 178  35]
 [221 222 142 ...  68 216  29]
 [134   0 164 ...  27 146 160]
 ...
 [135  11 194 ...   0 158 250]
 [ 85  73 105 ...  46  42 106]
 [ 48  73  22 ... 237 194  78]]
<class 'numpy.ndarray'>


#numba.cuda

In [2]:
from numba import cuda

ImportError: Numba needs NumPy 1.21 or less

In [3]:
cuda.detect()

NameError: name 'cuda' is not defined

In [70]:
# Importing NumPy
import numpy as np  

# Using numpy.random.rand() to generate an array of random numbers of size (2, 2)
random_array1 = np.random.rand(N)
radnom_array2 = np.random.rand(N)


In [71]:
ans =[]
for i in range(random_array1.size):
    ans.append(random_array1[i]*radnom_array2[i]) 
    

In [55]:
print(ans)

[0.049600174945875306, 0.04836575691397883, 0.0945811000522485, 0.19367070304946155, 0.2378207232994692, 0.015235294144712138, 0.04440892620195568, 0.017134004732664118, 0.056112696600885296, 0.02405702963315841, 0.1056482713860908, 0.5803322458977875, 0.3410060045419952, 0.8611443092637028, 0.22747057129413334, 0.10556723627977306, 0.11142799645355056, 0.31009838020032765, 0.014251867605170297, 0.09864654439669371, 0.2625767947646237, 0.04259395296053167, 0.26944149160866504, 0.6945953187058631, 0.1636652707965866, 0.5334470112128289, 0.6621028441820781, 0.2600436013013477, 0.3886417266075598, 0.5023589670127603]


In [37]:
print("Testing git")

Testing git
